# Multi-agent Collaboration

A single agent can usually perform well using a small set of tools to solve a specific task. However, even powerful models like GPT-4 may struggle when given many different tools to solve a variety of tasks.

One way to approach complicated tasks is through a _"divide-and-conquer"_ approach. Create a specialized agent for each task and route tasks to the correct _"expert"_.

The code from this notebook is adapted from the [LangGraph tutorial](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration). The ideas in this notebook is inspired by the paper [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155), by Wu, et. al.

The graph that we'll build will look something like the following diagram:

![multi_agent diagram](img/simple_multi_agent_diagram.png)

**Figure 1**: Image from [LangGraph](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration)

## Setup

Before we begin, please make sure you have setup the `.env` file in the project 
directory as described in [`README.md`](README.md).

Next, we will load in the necessary environment variables (e.g., API keys) for this notebook:

In [13]:
import os
from dotenv import load_dotenv

_ = load_dotenv()

assert os.environ.get("GOOGLE_API_KEY")
assert os.environ.get("TAVILY_API_KEY")

## Create Agents

The following helper function will help create agents. These agents will then be nodes in the graph.

In [11]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder,
)
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.tools import BaseTool
from langchain_core.runnables import RunnableSequence


def create_agent(
    model: BaseChatModel, 
    tools: list[BaseTool], 
    system_message: str
) -> RunnableSequence:
    """
    Create an agent.

    Args:
        model: The LLM that powers this agent.
        tools: List of tools to be called by the LLM.
        system_message: The system message passed to the LLM 
            to configure its behaviour.

    Returns:
        A chain that can be invoked.
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, "
                "collaborating with other assistants. "
                "Use the provided tools to progress "
                "towards answering the question. "
                "If you are unable to fully answer, "
                "that's OK, another assistant with "
                "different tools will help where "
                "you left off. Execute what you can "
                "to make progress. If you or any of "
                "the other assistants have the "
                "final answer or deliverable, "
                "prefix your response with FINAL ANSWER "
                "so the team knows to stop. "
                "You have access to the following tools: {tool_names}."
                "\n\n"
                "{system_message}",
            ),
            MessagesPlaceholder(
                variable_name="messages"
            ),
        ]
    )
    prompt = prompt.partial(
        system_message=system_message
    )
    prompt = prompt.partial(
        tool_names=", ".join(
            tool.name for tool in tools
        )
    )
    return (
        prompt 
        | model.bind_tools(tools)
    )

In [ ]:
from langgraph.graph import (
    START,
    END, 
    StateGraph, 
)


## Define tools

We will also define some tools that our agents will use in the future

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=3)